# 🔧 Calibration for v10

## คู่มือ Calibration ทั้งหมดสำหรับ Hybrid Grasp Detection v10

### 📋 รายการ Calibration
| # | Parameter | หน่วย | ค่าปัจจุบัน | วิธี Calibrate |
|---|-----------|------|------------|----------------|
| 1 | `PIXELS_PER_MM` | px/mm | 2.240 | วัดวัตถุที่รู้ขนาด |
| 2 | `DEPTH_Z_SCALE` | mm/unit | 63.2093 | วัดความสูงวัตถุหลายชิ้น |
| 3 | `HOMOGRAPHY_MATRIX` | - | (3x3) | 4-point calibration |
| 4 | `ROBOT_R_OFFSET` | degree | -25.55 | หา neutral angle |
| 5 | `Z_FLOOR` | mm | -64 | วัดจาก Dobot Studio |
| 6 | Gripper Calibration | mm↔angle | table | วัด gripper |

---

## 1️⃣ Imports & Setup

In [9]:
import sys
import cv2
import numpy as np
import time
import torch
import json
from datetime import datetime

sys.path.append('Depth-Anything-V2')

# Camera ID - แก้ตาม setup ของคุณ
CAMERA_ID = 2  # หรือ 1, 0

print("✓ Imports ready")
print(f"   Camera ID: {CAMERA_ID}")

✓ Imports ready
   Camera ID: 2


In [10]:
# Load Depth Model
from depth_anything_v2.dpt import DepthAnythingV2

DEPTH_MODEL_PATH = 'Depth-Anything-V2/checkpoints/depth_anything_v2_vits.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model_configs = {'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]}}
depth_model = DepthAnythingV2(**model_configs['vits'])
depth_model.load_state_dict(torch.load(DEPTH_MODEL_PATH, map_location='cpu', weights_only=True))
depth_model = depth_model.to(DEVICE).eval()
print(f"✅ Depth model on {DEVICE}")

✅ Depth model on cpu


---
# 📏 CALIBRATION 1: PIXELS_PER_MM
---

## วัตถุประสงค์
หาค่าแปลง pixel เป็น millimeter สำหรับกล้อง

## สิ่งที่ต้องเตรียม
- **วัตถุที่รู้ขนาดแน่นอน** เช่น:
  - ไม้บรรทัด
  - กล่องที่รู้ขนาด
  - บัตรเครดิต (85.6 x 53.98 mm)

## ขั้นตอน
1. วางวัตถุที่รู้ขนาดบนพื้นที่ทำงาน
2. **ลากเส้น** จากขอบหนึ่งไปอีกขอบ
3. กด **Enter** แล้วใส่ขนาดจริงเป็น **mm**
4. บันทึกค่า `PIXELS_PER_MM`

In [14]:
print("="*60)
print("📏 CALIBRATION: PIXELS_PER_MM")
print("="*60)
print("\n📋 ขั้นตอน:")
print("   1. วางวัตถุที่รู้ขนาด (เช่น ไม้บรรทัด) ในกล้อง")
print("   2. ลากเส้นจากขอบหนึ่งไปอีกขอบ (ตามแนวที่รู้ขนาด)")
print("   3. กด ENTER แล้วพิมพ์ขนาดจริง (mm)")
print("   4. กด Q เพื่อจบ\n")

drawing = False
start_pt, end_pt = None, None
px_dist = 0
calibration_samples = []

def ppm_callback(event, x, y, flags, param):
    global drawing, start_pt, end_pt, px_dist
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing, start_pt = True, (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        end_pt = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing, end_pt = False, (x, y)
        px_dist = np.sqrt((end_pt[0]-start_pt[0])**2 + (end_pt[1]-start_pt[1])**2)
        print(f"   📐 Distance: {px_dist:.1f} pixels")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('PIXELS_PER_MM Calibration')
cv2.setMouseCallback('PIXELS_PER_MM Calibration', ppm_callback)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    display = frame.copy()
    if start_pt and end_pt:
        cv2.line(display, start_pt, end_pt, (0,255,0), 2)
        mid = ((start_pt[0]+end_pt[0])//2, (start_pt[1]+end_pt[1])//2)
        cv2.putText(display, f"{px_dist:.1f}px", mid, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    
    # Status
    cv2.rectangle(display, (0,0), (640,50), (30,30,30), -1)
    cv2.putText(display, f"Samples: {len(calibration_samples)} | Draw line -> ENTER -> type mm | Q=Done", 
                (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    if calibration_samples:
        avg_ppm = np.mean([s['ppm'] for s in calibration_samples])
        cv2.putText(display, f"Current avg: {avg_ppm:.4f} px/mm", (10, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,0), 1)
    
    cv2.imshow('PIXELS_PER_MM Calibration', display)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    elif key == 13 and px_dist > 0:  # Enter
        try:
            mm = float(input("   📏 ใส่ขนาดจริง (mm): "))
            if mm > 0:
                ppm = px_dist / mm
                calibration_samples.append({'px': px_dist, 'mm': mm, 'ppm': ppm})
                print(f"   ✅ Sample {len(calibration_samples)}: {px_dist:.1f}px / {mm:.1f}mm = {ppm:.4f} px/mm")
                start_pt, end_pt, px_dist = None, None, 0
        except ValueError:
            print("   ❌ กรุณาใส่ตัวเลข")

cap.release()
cv2.destroyAllWindows()

if calibration_samples:
    PIXELS_PER_MM = np.mean([s['ppm'] for s in calibration_samples])
    print(f"\n" + "="*60)
    print(f"✅ PIXELS_PER_MM = {PIXELS_PER_MM:.4f}")
    print(f"   (จาก {len(calibration_samples)} samples)")
    print("="*60)
else:
    PIXELS_PER_MM = 2.240
    print(f"\n⚠️ ใช้ค่าเดิม: PIXELS_PER_MM = {PIXELS_PER_MM}")

📏 CALIBRATION: PIXELS_PER_MM

📋 ขั้นตอน:
   1. วางวัตถุที่รู้ขนาด (เช่น ไม้บรรทัด) ในกล้อง
   2. ลากเส้นจากขอบหนึ่งไปอีกขอบ (ตามแนวที่รู้ขนาด)
   3. กด ENTER แล้วพิมพ์ขนาดจริง (mm)
   4. กด Q เพื่อจบ

   📐 Distance: 70.0 pixels
   ✅ Sample 1: 70.0px / 28.0mm = 2.5003 px/mm
   📐 Distance: 123.0 pixels
   ✅ Sample 2: 123.0px / 50.0mm = 2.4603 px/mm
   📐 Distance: 62.0 pixels
   📐 Distance: 67.0 pixels
   ✅ Sample 3: 67.0px / 20.0mm = 3.3504 px/mm

✅ PIXELS_PER_MM = 2.7703
   (จาก 3 samples)


---
# 📐 CALIBRATION 2: DEPTH_Z_SCALE
---

## วัตถุประสงค์
หาค่าแปลง depth value เป็น mm ความสูง

## สิ่งที่ต้องเตรียม
- **วัตถุหลายชิ้นที่รู้ความสูง** เช่น:
  - กล่องสูง 30mm
  - หนังสือหนา 25mm
  - แก้วน้ำสูง 100mm

## ขั้นตอน
1. **เคลียร์พื้นที่** แล้วกด **C** เพื่อ calibrate floor
2. **วางวัตถุ** ที่รู้ความสูง
3. **คลิก** บนวัตถุ แล้วใส่ความสูงจริง (mm)
4. ทำซ้ำกับวัตถุหลายชิ้น
5. กด **S** เพื่อดูผลลัพธ์

In [15]:
print("="*60)
print("📐 CALIBRATION: DEPTH_Z_SCALE")
print("="*60)
print("\n📋 ขั้นตอน:")
print("   1. เคลียร์พื้นที่ให้ว่าง แล้วกด C เพื่อ calibrate floor")
print("   2. วางวัตถุที่รู้ความสูง (mm) ลงไป")
print("   3. คลิกบนวัตถุ แล้วพิมพ์ความสูงจริง (mm)")
print("   4. ทำซ้ำ 3-5 ครั้งกับวัตถุต่างความสูง")
print("   5. กด S เพื่อดูผลลัพธ์ | Q เพื่อจบ\n")

depth_samples = []
floor_depth = None
click_x, click_y = None, None
current_depth_map = None

def depth_cal_cb(event, x, y, flags, param):
    global click_x, click_y
    if event == cv2.EVENT_LBUTTONDOWN:
        click_x, click_y = x, y

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('DEPTH_Z_SCALE Calibration')
cv2.setMouseCallback('DEPTH_Z_SCALE Calibration', depth_cal_cb)
fc = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    fc += 1
    if fc % 5 == 0:
        current_depth_map = depth_model.infer_image(frame)
    
    display = frame.copy()
    
    # Depth overlay
    if current_depth_map is not None:
        depth_norm = cv2.normalize(current_depth_map, None, 0, 255, cv2.NORM_MINMAX)
        depth_color = cv2.applyColorMap(depth_norm.astype(np.uint8), cv2.COLORMAP_JET)
        depth_resized = cv2.resize(depth_color, (frame.shape[1], frame.shape[0]))
        display = cv2.addWeighted(display, 0.6, depth_resized, 0.4, 0)
    
    # Status
    cv2.rectangle(display, (0,0), (640,50), (30,30,30), -1)
    floor_str = f"Floor: {floor_depth:.4f}" if floor_depth else "Floor: NOT SET (press C)"
    cv2.putText(display, f"{floor_str} | Samples: {len(depth_samples)}", 
                (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    if depth_samples:
        avg_scale = np.mean([s['scale'] for s in depth_samples])
        cv2.putText(display, f"Avg scale: {avg_scale:.4f} mm/unit", (10, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,0), 1)
    
    cv2.imshow('DEPTH_Z_SCALE Calibration', display)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    elif key == ord('c') and current_depth_map is not None:
        h, w = current_depth_map.shape
        floor_depth = np.median(current_depth_map[h//3:2*h//3, w//3:2*w//3])
        print(f"   ✅ Floor calibrated: {floor_depth:.4f}")
    elif key == ord('s') and depth_samples:
        avg_scale = np.mean([s['scale'] for s in depth_samples])
        print(f"\n   📊 Samples: {len(depth_samples)}")
        for i, s in enumerate(depth_samples):
            print(f"      {i+1}. Height={s['height_mm']}mm → Scale={s['scale']:.4f}")
        print(f"\n   ✅ DEPTH_Z_SCALE = {avg_scale:.4f}")
    
    # Handle click
    if click_x is not None and current_depth_map is not None and floor_depth is not None:
        dh, dw = current_depth_map.shape
        fh, fw = frame.shape[:2]
        dx = int(click_x * dw / fw)
        dy = int(click_y * dh / fh)
        
        region = current_depth_map[max(0,dy-10):dy+10, max(0,dx-10):dx+10]
        if region.size > 0:
            raw_diff = np.median(region) - floor_depth
            if raw_diff > 0.01:
                try:
                    h_mm = float(input(f"   📏 ความสูงวัตถุที่ ({click_x},{click_y}) (mm): "))
                    if h_mm > 0:
                        scale = h_mm / raw_diff
                        depth_samples.append({'height_mm': h_mm, 'raw_diff': raw_diff, 'scale': scale})
                        print(f"   ✅ Sample {len(depth_samples)}: {h_mm}mm / {raw_diff:.4f} = {scale:.4f}")
                except ValueError:
                    print("   ❌ กรุณาใส่ตัวเลข")
            else:
                print("   ⚠️ จุดนี้เท่ากับพื้น ลองคลิกบนวัตถุ")
        click_x, click_y = None, None

cap.release()
cv2.destroyAllWindows()

if depth_samples:
    DEPTH_Z_SCALE = np.mean([s['scale'] for s in depth_samples])
    print(f"\n" + "="*60)
    print(f"✅ DEPTH_Z_SCALE = {DEPTH_Z_SCALE:.4f}")
    print(f"   (จาก {len(depth_samples)} samples)")
    print("="*60)
else:
    DEPTH_Z_SCALE = 63.2093
    print(f"\n⚠️ ใช้ค่าเดิม: DEPTH_Z_SCALE = {DEPTH_Z_SCALE}")

📐 CALIBRATION: DEPTH_Z_SCALE

📋 ขั้นตอน:
   1. เคลียร์พื้นที่ให้ว่าง แล้วกด C เพื่อ calibrate floor
   2. วางวัตถุที่รู้ความสูง (mm) ลงไป
   3. คลิกบนวัตถุ แล้วพิมพ์ความสูงจริง (mm)
   4. ทำซ้ำ 3-5 ครั้งกับวัตถุต่างความสูง
   5. กด S เพื่อดูผลลัพธ์ | Q เพื่อจบ

   ✅ Floor calibrated: 2.3297
   ✅ Sample 1: 12.0mm / 1.2423 = 9.6598
   ✅ Sample 2: 24.0mm / 1.1157 = 21.5108
   ✅ Sample 3: 36.0mm / 1.2388 = 29.0614
   ✅ Sample 4: 48.0mm / 1.2547 = 38.2564
   ✅ Sample 5: 60.0mm / 1.1168 = 53.7253
   ✅ Sample 6: 72.0mm / 1.2360 = 58.2509
   ✅ Sample 7: 84.0mm / 1.3158 = 63.8377
   ✅ Sample 8: 96.0mm / 1.0818 = 88.7379

✅ DEPTH_Z_SCALE = 45.3800
   (จาก 8 samples)


---
# 🎯 CALIBRATION 3: HOMOGRAPHY_MATRIX
---

## วัตถุประสงค์
หา matrix แปลงพิกัด pixel → พิกัด robot

## สิ่งที่ต้องเตรียม
- **4 จุด marker** วางบนพื้นที่ทำงาน
- **Dobot Studio** เปิดอยู่ (Online mode)

## ขั้นตอน
1. วาง **4 markers** เป็นสี่เหลี่ยม
2. ใช้ **Dobot Studio** จด X,Y ของแต่ละจุด
3. **คลิก** บน marker ในกล้อง
4. ใส่ **X,Y** ที่จดไว้
5. ทำครบ 4 จุด

In [16]:
print("="*60)
print("🎯 CALIBRATION: HOMOGRAPHY_MATRIX")
print("="*60)
print("\n📋 ขั้นตอน:")
print("   1. วาง 4 markers เป็นสี่เหลี่ยมบนพื้นที่ทำงาน")
print("   2. ใช้ Dobot Studio (Online mode) จดพิกัด X,Y ของแต่ละจุด")
print("   3. คลิกบน marker ในกล้อง")
print("   4. พิมพ์พิกัด robot X,Y (mm)")
print("   5. ทำครบ 4 จุด | Q=Done\n")

pixel_points = []
robot_points = []
click_pos = None

def homo_cb(event, x, y, flags, param):
    global click_pos
    if event == cv2.EVENT_LBUTTONDOWN:
        click_pos = (x, y)

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('HOMOGRAPHY Calibration')
cv2.setMouseCallback('HOMOGRAPHY Calibration', homo_cb)

point_labels = ['A (TopLeft)', 'B (TopRight)', 'C (BottomRight)', 'D (BottomLeft)']

while cap.isOpened() and len(pixel_points) < 4:
    ret, frame = cap.read()
    if not ret: break
    
    display = frame.copy()
    
    # Draw existing points
    for i, pt in enumerate(pixel_points):
        cv2.circle(display, pt, 8, (0,0,255), -1)
        cv2.putText(display, point_labels[i].split()[0], (pt[0]+10, pt[1]), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    # Status
    cv2.rectangle(display, (0,0), (640,40), (30,30,30), -1)
    cv2.putText(display, f"Click on: {point_labels[len(pixel_points)]} | Points: {len(pixel_points)}/4", 
                (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    
    cv2.imshow('HOMOGRAPHY Calibration', display)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    
    if click_pos is not None:
        print(f"\n   📍 Point {point_labels[len(pixel_points)]}")
        print(f"      Pixel: ({click_pos[0]}, {click_pos[1]})")
        try:
            robot_x = float(input(f"      Robot X (mm): "))
            robot_y = float(input(f"      Robot Y (mm): "))
            pixel_points.append(click_pos)
            robot_points.append((robot_x, robot_y))
            print(f"      ✅ Added: pixel {click_pos} → robot ({robot_x}, {robot_y})")
        except ValueError:
            print("      ❌ กรุณาใส่ตัวเลข")
        click_pos = None

cap.release()
cv2.destroyAllWindows()

if len(pixel_points) == 4:
    src = np.array(pixel_points, dtype=np.float32)
    dst = np.array(robot_points, dtype=np.float32)
    HOMOGRAPHY_MATRIX, _ = cv2.findHomography(src, dst)
    
    print(f"\n" + "="*60)
    print("✅ HOMOGRAPHY_MATRIX =")
    print(f"{HOMOGRAPHY_MATRIX}")
    print("="*60)
    
    # Save to file
    np.save('homography_matrix.npy', HOMOGRAPHY_MATRIX)
    print(f"\n   💾 Saved to homography_matrix.npy")
else:
    print(f"\n⚠️ ต้องมี 4 จุด (มี {len(pixel_points)} จุด)")
    HOMOGRAPHY_MATRIX = None

🎯 CALIBRATION: HOMOGRAPHY_MATRIX

📋 ขั้นตอน:
   1. วาง 4 markers เป็นสี่เหลี่ยมบนพื้นที่ทำงาน
   2. ใช้ Dobot Studio (Online mode) จดพิกัด X,Y ของแต่ละจุด
   3. คลิกบน marker ในกล้อง
   4. พิมพ์พิกัด robot X,Y (mm)
   5. ทำครบ 4 จุด | Q=Done


   📍 Point A (TopLeft)
      Pixel: (330, 150)
      ✅ Added: pixel (330, 150) → robot (37.4, 41.58)

   📍 Point B (TopRight)
      Pixel: (329, 398)
      ✅ Added: pixel (329, 398) → robot (-34.01, 43.43)

   📍 Point C (BottomRight)
      Pixel: (142, 147)
      ✅ Added: pixel (142, 147) → robot (40.57, 115.46)

   📍 Point D (BottomLeft)
      Pixel: (144, 398)
      ✅ Added: pixel (144, 398) → robot (-38.19, 115.13)

✅ HOMOGRAPHY_MATRIX =
[[ 6.96573861e-03 -3.43200376e-01  9.34739573e+01]
 [-4.04493204e-01  1.00366138e-02  1.81220209e+02]
 [ 5.27846332e-04  6.73642275e-05  1.00000000e+00]]

   💾 Saved to homography_matrix.npy


---
# 🔄 CALIBRATION 4: ROBOT_R_OFFSET
---

## วัตถุประสงค์
หาค่า R (rotation) ของ robot เมื่อ gripper อยู่ในตำแหน่ง neutral (0°)

## ขั้นตอน
1. ใช้ **Dobot Studio** (Online mode)
2. หมุน gripper ให้ขนานกับกล้อง (gripper ชี้ตรง)
3. จดค่า **R** ที่แสดง

In [ ]:
print("="*60)
print("🔄 CALIBRATION: ROBOT_R_OFFSET")
print("="*60)
print("\n📋 ขั้นตอน:")
print("   1. เปิด Dobot Studio (Online mode)")
print("   2. หมุน gripper ให้ขนานกับกล้อง")
print("   3. จดค่า R ที่แสดง\n")

try:
    ROBOT_R_OFFSET = float(input("   📐 ใส่ค่า R offset (degree): "))
    print(f"\n" + "="*60)
    print(f"✅ ROBOT_R_OFFSET = {ROBOT_R_OFFSET}")
    print("="*60)
except ValueError:
    ROBOT_R_OFFSET = -25.55
    print(f"\n⚠️ ใช้ค่าเดิม: ROBOT_R_OFFSET = {ROBOT_R_OFFSET}")

---
# 📍 CALIBRATION 5: Z_FLOOR
---

## วัตถุประสงค์
หาค่า Z เมื่อ gripper แตะพื้น

## ขั้นตอน
1. ใช้ **Dobot Studio** (Online mode)
2. เลื่อน gripper ลงจนเกือบแตะพื้น
3. จดค่า **Z**

In [ ]:
print("="*60)
print("📍 CALIBRATION: Z_FLOOR")
print("="*60)
print("\n📋 ขั้นตอน:")
print("   1. เปิด Dobot Studio (Online mode)")
print("   2. เลื่อน gripper ลงจนปลายเกือบแตะพื้น")
print("   3. จดค่า Z ที่แสดง\n")

try:
    Z_FLOOR = float(input("   📐 ใส่ค่า Z floor (mm): "))
    Z_SAFE = Z_FLOOR + 24  # Safe height above floor
    Z_APPROACH = Z_FLOOR + 9  # Approach height
    print(f"\n" + "="*60)
    print(f"✅ Z_FLOOR = {Z_FLOOR}")
    print(f"   Z_SAFE = {Z_SAFE} (floor + 24mm)")
    print(f"   Z_APPROACH = {Z_APPROACH} (floor + 9mm)")
    print("="*60)
except ValueError:
    Z_FLOOR = -64
    Z_SAFE = -40
    Z_APPROACH = -55
    print(f"\n⚠️ ใช้ค่าเดิม: Z_FLOOR = {Z_FLOOR}")

---
# 🦾 CALIBRATION 6: GRIPPER
---

## วัตถุประสงค์
หาความสัมพันธ์ระหว่าง servo angle กับ gripper width

## สิ่งที่ต้องเตรียม
- **ไม้บรรทัด** หรือ caliper
- **ESP32** Gripper controller connected

## ขั้นตอน
1. ส่งคำสั่ง angle ไปที่ gripper
2. วัด width ด้วยไม้บรรทัด
3. บันทึก angle ↔ width

In [ ]:
import serial

print("="*60)
print("🦾 CALIBRATION: GRIPPER")
print("="*60)

ESP32_PORT = input("   🔌 ESP32 Port (เช่น COM9): ") or 'COM9'

try:
    ser = serial.Serial(ESP32_PORT, 115200, timeout=2)
    time.sleep(2)
    print(f"   ✅ Connected to {ESP32_PORT}")
    
    gripper_samples = []
    
    print("\n📋 ขั้นตอน:")
    print("   1. พิมพ์ angle (เช่น 22, 50, 96)")
    print("   2. วัด width ด้วยไม้บรรทัด (mm)")
    print("   3. พิมพ์ 'done' เมื่อเสร็จ\n")
    
    while True:
        cmd = input("   📏 Angle (หรือ 'done'): ")
        if cmd.lower() == 'done':
            break
        try:
            angle = int(cmd)
            ser.write(f'G{angle}\n'.encode())
            time.sleep(1)
            width = float(input(f"      Width ที่ angle {angle}° (mm): "))
            gripper_samples.append({'angle': angle, 'width': width})
            print(f"      ✅ {angle}° → {width}mm")
        except ValueError:
            print("      ❌ กรุณาใส่ตัวเลข")
    
    ser.close()
    
    if gripper_samples:
        gripper_samples.sort(key=lambda x: x['angle'])
        print(f"\n" + "="*60)
        print("✅ GRIPPER CALIBRATION TABLE")
        print("   Angle (°) | Width (mm)")
        print("   ----------|-----------")
        for s in gripper_samples:
            print(f"   {s['angle']:8} | {s['width']:.1f}")
        print("="*60)
        
        CALIB_ANGLES = np.array([s['angle'] for s in gripper_samples])
        CALIB_WIDTHS = np.array([s['width'] for s in gripper_samples])
except Exception as e:
    print(f"   ❌ Error: {e}")
    print("\n⚠️ ใช้ค่า default")
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])

---
# 💾 SAVE ALL CALIBRATION
---

In [8]:
print("="*60)
print("💾 CALIBRATION SUMMARY")
print("="*60)

# Collect all values
calibration_data = {
    'PIXELS_PER_MM': PIXELS_PER_MM,
    'DEPTH_Z_SCALE': DEPTH_Z_SCALE,
    'ROBOT_R_OFFSET': ROBOT_R_OFFSET,
    'Z_FLOOR': Z_FLOOR,
    'Z_SAFE': Z_SAFE,
    'Z_APPROACH': Z_APPROACH,
    'CAMERA_ID': CAMERA_ID,
    'calibration_date': datetime.now().isoformat()
}

print("\n📋 ค่าทั้งหมด:")
print(f"   PIXELS_PER_MM = {PIXELS_PER_MM:.4f}")
print(f"   DEPTH_Z_SCALE = {DEPTH_Z_SCALE:.4f}")
print(f"   ROBOT_R_OFFSET = {ROBOT_R_OFFSET}")
print(f"   Z_FLOOR = {Z_FLOOR}")
print(f"   Z_SAFE = {Z_SAFE}")
print(f"   Z_APPROACH = {Z_APPROACH}")

# Save to JSON
with open('calibration_v10.json', 'w') as f:
    json.dump(calibration_data, f, indent=2)
print(f"\n💾 Saved to calibration_v10.json")

# Print code to copy
print("\n" + "="*60)
print("📋 COPY THIS TO v10 NOTEBOOK:")
print("="*60)
print(f'''
# === CALIBRATED VALUES ===
PIXELS_PER_MM = {PIXELS_PER_MM:.4f}
DEPTH_Z_SCALE = {DEPTH_Z_SCALE:.4f}
CAMERA_HEIGHT_MM = 630

# === Robot R Rotation ===
ROBOT_R_OFFSET = {ROBOT_R_OFFSET}

# === Z Heights ===
Z_FLOOR = {Z_FLOOR}
Z_SAFE = {Z_SAFE}
Z_APPROACH = {Z_APPROACH}
''')

💾 CALIBRATION SUMMARY


NameError: name 'PIXELS_PER_MM' is not defined

In [ ]:
# Print Homography Matrix code
if HOMOGRAPHY_MATRIX is not None:
    print("\n📋 HOMOGRAPHY_MATRIX:")
    print(f"HOMOGRAPHY_MATRIX = np.array([")
    for row in HOMOGRAPHY_MATRIX:
        print(f"    [{row[0]}, {row[1]}, {row[2]}],")
    print("], dtype=np.float32)")